In [1]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split

2023-06-06 14:47:21.015074: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 14:47:21.051565: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 14:47:21.052163: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 14:47:22.065123: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

1331


# Handmade NN

In [3]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

# def build_model_convolutional(input_dim):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
#         tf.keras.layers.Dense(7*7*128, activation='relu'),
#         tf.keras.layers.Reshape((7, 7, 128)),
#         tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
#         tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
#     ])
#     return model

In [4]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [6]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
# model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')
# model_conv.compile(optimizer='adam', loss='mse')

2023-06-06 14:47:35.998985: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 14:47:36.000358: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 14:47:36.001650: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [7]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=10)

Epoch 1/10
32/32 [==============================] - 2s 28ms/step - loss: 1260087.8750
Epoch 2/10
32/32 [==============================] - 1s 27ms/step - loss: 1302.4532
Epoch 3/10
32/32 [==============================] - 1s 27ms/step - loss: 1300.7681
Epoch 4/10
32/32 [==============================] - 1s 27ms/step - loss: 1300.7672
Epoch 5/10
32/32 [==============================] - 1s 27ms/step - loss: 1300.7671
Epoch 6/10
32/32 [==============================] - 1s 27ms/step - loss: 1300.7665
Epoch 7/10
32/32 [==============================] - 1s 27ms/step - loss: 1300.7662
Epoch 8/10
32/32 [==============================] - 1s 27ms/step - loss: 1300.7657
Epoch 9/10
32/32 [==============================] - 1s 28ms/step - loss: 1300.7653
Epoch 10/10
32/32 [==============================] - 1s 28ms/step - loss: 1300.7645


In [8]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=10)

Epoch 1/10


2023-06-06 14:47:56.871073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 14:47:56.873172: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 14:47:56.874807: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - 3s 14ms/step - loss: 1300.7408
Epoch 2/10
32/32 [==============================] - 0s 14ms/step - loss: 1300.5596
Epoch 3/10
32/32 [==============================] - 0s 14ms/step - loss: 1300.3154
Epoch 4/10
32/32 [==============================] - 0s 14ms/step - loss: 1300.1185
Epoch 5/10
32/32 [==============================] - 0s 14ms/step - loss: 1300.0643
Epoch 6/10
32/32 [==============================] - 0s 14ms/step - loss: 1299.9861
Epoch 7/10
32/32 [==============================] - 0s 14ms/step - loss: 1299.8282
Epoch 8/10
32/32 [==============================] - 0s 14ms/step - loss: 1299.7389
Epoch 9/10
32/32 [==============================] - 0s 14ms/step - loss: 1299.6620
Epoch 10/10
32/32 [==============================] - 0s 14ms/step - loss: 1299.5140


In [9]:
from sklearn.metrics import mean_squared_error as mse
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
# Y_conv_predicted = model_conv.predict(np.array(X_test))
print(mse(Y_predicted, Y_test_flat))
print(mse(Y_ann_predicted, Y_test_flat))

11/11 [==============================] - 1s 3ms/step


2023-06-06 14:48:23.769294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 14:48:23.771385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 14:48:23.773267: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

11/11 [==============================] - 0s 4ms/step
89.83474161464166
90.26840105610201


In [10]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
Y_predicted

[array([[ 5.43421786e-03,  5.92146721e-03,  2.31136661e-03,
         -5.94113953e-04,  2.16544606e-04,  8.08021519e-03,
         -7.22180121e-04, -1.98379345e-03, -8.63410998e-04,
         -1.01033971e-03,  1.43521028e-02,  7.61359371e-03,
          3.52325942e-03,  6.17416948e-03],
        [-6.10079616e-04,  1.37829361e-02,  2.33449601e-03,
         -3.12759494e-03,  1.27893835e-02,  7.90172722e-04,
          1.19026238e-03,  7.45257176e-03,  9.56564676e-03,
          8.86872970e-03,  8.36688560e-03,  9.20810364e-03,
          1.00508798e-02,  4.35811542e-02],
        [ 1.03571266e-03, -1.45713042e-04,  1.58096850e-03,
          7.46793300e-03, -4.10617329e-04,  7.22300820e-03,
          8.46051052e-03,  1.33676268e-02,  1.02104712e-03,
          7.99113233e-03,  3.10104480e-03,  2.04866007e-03,
          5.78217488e-03,  2.17061695e-02],
        [ 6.44712336e-03,  3.30988271e-03, -1.44155510e-03,
          9.21945274e-03,  9.10255872e-03,  5.16032102e-03,
          8.71745870e-05,  8

In [11]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
Y_ann_predicted

[array([[-0.00466227, -0.0071553 ,  0.0005091 ,  0.00528793,  0.00217099,
          0.00860811, -0.00189922,  0.00578256, -0.00246265,  0.0013653 ,
         -0.00338636,  0.01084066, -0.00347241,  0.00720475],
        [ 0.00472163,  0.00279007,  0.00893801, -0.00249971, -0.0056039 ,
         -0.00823032, -0.00682782, -0.00576447, -0.00742619, -0.00569975,
         -0.01401022,  0.00381219,  0.00092044,  0.00069328],
        [-0.00327596, -0.0047935 , -0.01027456,  0.00227634,  0.0026548 ,
          0.00783615,  0.00316285,  0.00193876,  0.00547022,  0.01027158,
          0.00348459, -0.00251532,  0.00102801, -0.00030671],
        [ 0.00533858, -0.01019011,  0.00436373, -0.00128747, -0.00162924,
          0.00482508, -0.0027048 , -0.00127156, -0.0032173 ,  0.0089633 ,
          0.00113427,  0.00622453,  0.00427039,  0.00491235],
        [ 0.00318793, -0.00416687, -0.00478528, -0.00050036,  0.00681598,
          0.01015369,  0.00646314, -0.00448547,  0.00147688,  0.00387811,
          0.

In [ ]:
print(Y_conv_predicted)

# AutoKeras

In [12]:
import autokeras as ak
import numpy as np

In [13]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [ ]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=5000
)
reg.fit(train_set,epochs=10, validation_split=0.15)
predicted_y = reg.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)|
print(reg.evaluate(test_set))


Trial 17 Complete [00h 00m 11s]
val_loss: 27.22876739501953

Best val_loss So Far: 27.22876739501953
Total elapsed time: 00h 02m 40s

Search: Running Trial #18

Value             |Best Value So Far |Hyperparameter
False             |False             |structured_data_block_1/normalize
True              |True              |structured_data_block_1/dense_block_1/use_batchnorm
3                 |3                 |structured_data_block_1/dense_block_1/num_layers
128               |128               |structured_data_block_1/dense_block_1/units_0
0                 |0                 |structured_data_block_1/dense_block_1/dropout
512               |512               |structured_data_block_1/dense_block_1/units_1
0.5               |0.5               |regression_head_1/dropout
adam              |adam              |optimizer
0.0001            |0.001             |learning_rate
256               |256               |structured_data_block_1/dense_block_1/units_2



In [27]:
regressor = ak.StructuredDataRegressor(max_trials=100, overwrite=True)
regressor.fit(train_set, epochs=300, validation_split=0.15)
predicted_y = regressor.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(regressor.evaluate(test_set))

Trial 19 Complete [00h 00m 10s]
val_loss: 12040.6640625

Best val_loss So Far: 11934.3232421875
Total elapsed time: 00h 08m 10s

Search: Running Trial #20

Value             |Best Value So Far |Hyperparameter
False             |True              |structured_data_block_1/normalize
True              |False             |structured_data_block_1/dense_block_1/use_batchnorm
2                 |2                 |structured_data_block_1/dense_block_1/num_layers
64                |64                |structured_data_block_1/dense_block_1/units_0
0.5               |0                 |structured_data_block_1/dense_block_1/dropout
32                |256               |structured_data_block_1/dense_block_1/units_1
0                 |0                 |regression_head_1/dropout
adam              |adam              |optimizer
0.001             |0.001             |learning_rate
512               |32                |structured_data_block_1/dense_block_1/units_2



2023-06-06 14:46:40.736411: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_30' with dtype double and shape [998,196]
	 [[{{node Placeholder/_30}}]]
2023-06-06 14:46:40.736803: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_30' with dtype double and shape [998,196]
	 [[{{node Placeholder/_30}}]]
2023-06-06 14:46:40.849102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_29' with dtype double and 

2023-06-06 14:46:42.031405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_21' with dtype double and shape [998,128]
	 [[{{node Placeholder/_21}}]]
2023-06-06 14:46:42.031771: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_22' with dtype double and shape [998,196]
	 [[{{node Placeholder/_22}}]]
2023-06-06 14:46:42.136092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_30' with dtype double and 

2023-06-06 14:46:43.238756: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_30' with dtype double and shape [998,196]
	 [[{{node Placeholder/_30}}]]
2023-06-06 14:46:43.239120: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_22' with dtype double and shape [998,196]
	 [[{{node Placeholder/_22}}]]
2023-06-06 14:46:43.335338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_30' with dtype double and 

KeyboardInterrupt: 

In [ ]:
model = reg.export_model()
model.summary()

In [ ]:
model_regressor = regressor.export_model()
model_regressor.summary()

In [ ]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

# Autogoal